In [809]:
class convertCVtoText:
    @staticmethod
    def startConversion(fileName):
        import PyPDF2
        pdfFileObj = open(fileName,'rb')     #'rb' for read binary mode
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        noOfPages=pdfReader.numPages
        text=""
        for pages in range(0,pdfReader.numPages):
            pageObj = pdfReader.getPage(pages)          #'9' is the page number
            text+=pageObj.extractText()
        return text

In [810]:
class regularExp:
    @staticmethod
    def normalizeText(text):
        norText=""
        returnText=""
        
        import re
        norText+= re.sub(r'[^a-zA-Z ]',r' ',text)
        returnText+=re.sub(' +',' ',norText)
        norText=""
        norText+=re.sub(r'([A-Z])', lambda pat: pat.group(1).lower(), returnText)
        return norText
    @staticmethod
    def Stemmer(text):
        from nltk.stem import PorterStemmer
        import nltk
        words=nltk.word_tokenize(text)
        stemmer = PorterStemmer()
        singles = [stemmer.stem(word) for word in words]
        #print(singles)
        singles=list(set(singles))
        #print(singles)
        stemmedText=" ".join(singles)
        #print(stemmedText)
        return stemmedText
       
                
        
        

In [811]:
class NLTKHelper:
    @staticmethod
    def findDocumentMatrix(totalCVText,minFrequency,vocab):
        from sklearn.feature_extraction.text import CountVectorizer
        if vocab is None:
            vectorizer=CountVectorizer(stop_words='english',min_df=minFrequency)
        elif minFrequency is None:
            vectorizer=CountVectorizer(stop_words='english',vocabulary=vocab)
        else:
            vectorizer=CountVectorizer(stop_words='english')
        documentMatrix=vectorizer.fit_transform(totalCVText)
        vocabulary=vectorizer.vocabulary_
        return documentMatrix,vocabulary     

In [812]:
class CV:
     def __init__(self,path):
            self.fileName=path
            self.text=convertCVtoText.startConversion(self.fileName)
            self.cleanText=regularExp.normalizeText(self.text)
            self.stemedText=regularExp.Stemmer(self.cleanText)
            self.featureVector=[]
            #self.cleanTextNew=regularExp.normalizeText(self.stemedText)


In [813]:
class relevantWords:
    def __init__(self):
        self.category=["role","language","softwarePackage","experienceSkill","os"]
        self.jobs=["database Engineer","network engineer","software tester","web designer"]
        self.role=[]
        self.language=[]
        self.softwarePackage=[]
        self.experienceSkill=[]
        self.os=[]
        self.categoryDictionary={}
        self.relevantDictionary={}
        self.relevantWordList=[]
    def enterWords(self,category,words):
        if (category in self.category):
            if(category==self.category[0]):
                for word in words:
                    self.role.append(word)
            elif(category==self.category[1]):
                for word in words:
                    self.language.append(word)
            elif(category==self.category[2]):
                for word in words:
                    self.softwarePackage.append(word)
            elif(category==self.category[3]):
                for word in words:
                    self.experienceSkill.append(word)
            elif(category==self.category[4]):
                for word in words:
                    self.os.append(word)
        else:
            print("category not matched")
    def updateDict(self,jobName):
        tempDict={}
        tempDict.update({self.category[0]:self.role})
        tempDict.update({self.category[1]:self.language})
        tempDict.update({self.category[2]:self.softwarePackage})
        tempDict.update({self.category[3]:self.experienceSkill})
        tempDict.update({self.category[4]:self.os})
        self.role=[]
        self.language=[]
        self.softwarePackage=[]
        self.experienceSkill=[]
        self.os=[]
#         if(not(self.readfileMode("relevantDictionary")) is None):
#             self.relevantDictionary=self.readfileMode("relevantDictionary")
        self.relevantDictionary.update({jobName:tempDict})
#         self.writefileMode("relevantDictionary",self.relevantDictionary)
    def readfileMode(self,fileName):
        try:
            with open(fileName+".json","r") as file:
                data=json.load(file)
            file.close()
            return data
        except Exception as e:
            print("error in filemode.\n Error is"+str(e))
            return None
    def writefileMode(self,fileName,data):
        try:
            with open(fileName+".json","w") as file:
                json.dump(data,file)
            file.close()
        except Exception as e:
            print("error in filemode.\n Error is"+str(e))
    def getRelevantWords(self,source=None):
        relevantWords=[]
        if(source is None):
            source=self.relevantDictionary
        for key,value in source.items():
            for key1,value1 in value.items():
                for word in value1:
                    relevantWords.append(word)
        return set(relevantWords)
        

In [814]:
rv=relevantWords()


In [815]:
rv.enterWords("role",["communication","network","server","client"])
rv.enterWords("language",["python","bash","php","c++","c"])
rv.enterWords("softwarePackage",[])
rv.enterWords("experienceSkill",["lan","cloud","wan","man","gan","cyber","security","dns","iot","virtulization"])
rv.enterWords("os",["unix","windows","linux"])
rv.updateDict("network enginer")


In [816]:
class CVManager:
    def __init__(self):
        self.CVAdded=None
        #self.database=databaseHandler()
        self.CVList=[]
        self.CVFileName=[]
        self.fileNamesWithPath=[]
        self.CVTextColl=[]
        self.CVTextCollTest=[]
        self.documentMatrix=[]
        self.vocabulary={}
        self.documentMatrixTest=[]
        self.vocabularyTest={}
    def list_CVs(self,rootPath):
        import os,json
#             if(os.path.exists("CVNameList.json")):
#                 print("here")
#                 with open("CVNameList.json","r") as file:
#                     self.fileNamesWithPath=json.load(file)
#             else:   
        for root, dirs, files in os.walk(rootPath):
            for name in files:
                self.CVFileName.append(name)
                self.fileNamesWithPath.append(os.path.join(root, name))
#                 with open("CVNameList.json","w") as file:
#                     json.dump(self.fileNamesWithPath,file)
#                 file.close()
               
    def collectCV(self):
            for cvPath in self.fileNamesWithPath:
                try:
                    newCV=CV(cvPath)
                    self.CVList.append(newCV)
                    #print("%s named CV is already in database"%cvName)
                except Exception as e:
                    print(cvPath)
                    print("in collection of CV \t"+str(e))
            
    def collectCVText(self):
        for cv in self.CVList:
            self.CVTextColl.append(cv.cleanText) 
            self.CVTextCollTest.append(cv.stemedText) 
    def findDocumentMatrix(self,minFrequency,vocab):
        import json
        try:
                self.collectCVText()
                self.documentMatrix,self.vocabulary=NLTKHelper.findDocumentMatrix(self.CVTextColl,minFrequency,vocab)
                self.documentMatrixTest,self.vocabularyTest=NLTKHelper.findDocumentMatrix(self.CVTextCollTest,minFrequency,vocab)
#                 with open("wholeCVText.json","w") as file:
#                     json.dump(self.CVTextColl,file)
#                 file.close()
#                 with open("wholeCVTextTest.json","w") as file:
#                     json.dump(self.CVTextCollTest,file)
#                 file.close()
#             else:
#                 with open("wholeCVText.json","r") as file:
#                     self.CVTextColl=json.load(file)
#                 file.close()
#                 with open("wholeCVTextTest.json","r") as file:
#                     self.CVTextCollTest=json.load(file)
#                 file.close()
#                 self.documentMatrix,self.vocabulary=NLTKHelper.findDocumentMatrix(self.CVTextColl,minFrequency,vocab)
#                 self.documentMatrixTest,self.vocabularyTest=NLTKHelper.findDocumentMatrix(self.CVTextCollTest,minFrequency,vocab)
        except Exception as e:
            print("in reading/writing of saved files \t"+str(e))
            import os
            try:
                os.remove("CVNameList.json")
            except Exception as e:
                print("in deleting the file \t"+str(e))
    def normalizeFeatureVector(self):
        from sklearn.preprocessing import normalize
        import numpy as np
        vectorValue=[]
        for key in self.vocabularyTest:
            vectorValue.append(self.featureVectorDict[key])
        datas = np.asarray(vectorValue, dtype=np.float)
        datas =normalize(datas, norm='l2')
        #converting numpy.array to list
        for r in datas:
            for column in r:
                self.normalizedVectorValue.append(column)
        print(self.normalizedVectorValue)
                
            
        
        

In [817]:
manager=CVManager()

In [818]:
manager.list_CVs("CV coll\\sagar\\network engineer")


In [819]:
manager.collectCV()

In [820]:
#use this function to test the feature vector given by relevant words
#manager.findDocumentMatrix(None,rv.getRelevantWords())
#use this function to find the most ferquently words as given in function
#it will help to find the relevant word for that post
manager.findDocumentMatrix(30,None)

In [821]:
print(len(manager.vocabulary))

68


In [822]:
print(len(manager.vocabularyTest))

91


In [823]:
print(manager.documentMatrix.toarray())



[[ 0  1  1 ...,  1  1  0]
 [ 0  0  0 ...,  0  0  0]
 [ 0  0  0 ...,  0  0  0]
 ..., 
 [ 1  0  1 ...,  2  1  0]
 [ 3  0  1 ...,  2  0 10]
 [ 0  0  1 ...,  2  1  6]]


In [824]:
manager.vocabulary

{'access': 0,
 'active': 1,
 'additional': 2,
 'authorized': 3,
 'center': 4,
 'cisco': 5,
 'com': 6,
 'communication': 7,
 'computer': 8,
 'configuration': 9,
 'customer': 10,
 'data': 11,
 'design': 12,
 'devices': 13,
 'education': 14,
 'email': 15,
 'employer': 16,
 'engineer': 17,
 'environment': 18,
 'equipment': 19,
 'experience': 20,
 'hardware': 21,
 'implementation': 22,
 'including': 23,
 'information': 24,
 'infrastructure': 25,
 'installation': 26,
 'ip': 27,
 'issues': 28,
 'june': 29,
 'lan': 30,
 'maintenance': 31,
 'management': 32,
 'manager': 33,
 'microsoft': 34,
 'network': 35,
 'networking': 36,
 'networks': 37,
 'new': 38,
 'office': 39,
 'present': 40,
 'project': 41,
 'provided': 42,
 'responsible': 43,
 'routers': 44,
 'routing': 45,
 'security': 46,
 'server': 47,
 'servers': 48,
 'service': 49,
 'skills': 50,
 'software': 51,
 'support': 52,
 'switches': 53,
 'systems': 54,
 'team': 55,
 'technical': 56,
 'technologies': 57,
 'technology': 58,
 'troubleshoot